# Group Delay, Phase, and Dispersion in Discrete-Time Systems

This notebook completes the pole–zero sequence by explaining:

- Phase vs frequency  
- **Group delay** as the derivative of phase  
- Why linear phase ⇒ constant group delay  
- How poles and zeros shape group delay  
- What **dispersion** means in signals and waves  
- Why this matters in seismology, acoustics, and imaging  

This connects frequency-domain geometry to **time-domain waveform distortion**.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


## 1) Phase response and why it matters

For a frequency response:

$$
H(e^{j\omega}) = |H(e^{j\omega})| e^{j\phi(\omega)}
$$

The two terms of the right control **amplitude shaping**  and **timing and waveform shape**  respectively.

Even if two systems have the same magnitude response, different phase responses can produce very different time-domain outputs.


## 2) Group delay

The **group delay** is defined as:

\[
\tau_g(\omega) = -\frac{d\phi(\omega)}{d\omega}
\]

Interpretation:

- Measures the delay of a **narrowband wave packet** centered at \(\omega\)  
- Tells you how fast energy at each frequency propagates through the system  

If \(\tau_g(\omega)\) is constant:

- All frequencies are delayed by the same amount  
- No waveform distortion due to phase  


## 3) Linear phase and constant group delay

A discrete-time system has **linear phase** if:

\[
\phi(\omega) = -\omega n_0
\]

Then:

\[
\tau_g(\omega) = n_0 \quad \text{(constant)}
\]

This means:

- Pure time shift by \(n_0\) samples  
- No dispersion  
- Perfect waveform shape preservation  

This is why **linear-phase FIR filters** are so valuable.


## 4) Example: linear-phase FIR vs nonlinear-phase FIR

In [ ]:
# Linear-phase FIR (symmetric kernel)
b_lin = np.array([1, 2, 3, 2, 1], dtype=float)
b_lin = b_lin / np.sum(b_lin)

# Nonlinear-phase FIR (asymmetric)
b_non = np.array([1, 0, 0, 0, -1], dtype=float)

w = np.linspace(0, np.pi, 2048)
z = np.exp(1j*w)

def freq_response(b):
    H = sum(b[k] * z**(-k) for k in range(len(b)))
    return H

H_lin = freq_response(b_lin)
H_non = freq_response(b_non)

# Phase
phi_lin = np.unwrap(np.angle(H_lin))
phi_non = np.unwrap(np.angle(H_non))

# Group delay (numerical derivative)
tau_lin = -np.gradient(phi_lin, w)
tau_non = -np.gradient(phi_non, w)

# Plot phase
fig, ax = plt.subplots(2, 1, figsize=(8,8), sharex=True)

ax[0].plot(w, phi_lin, label="Linear-phase FIR")
ax[0].plot(w, phi_non, label="Nonlinear-phase FIR")
ax[0].set_title("Phase responses")
ax[0].set_ylabel("Phase (rad)")
ax[0].legend()

# Plot group delay
ax[1].plot(w, tau_lin, label="Linear-phase FIR")
ax[1].plot(w, tau_non, label="Nonlinear-phase FIR")
ax[1].set_title("Group delay")
ax[1].set_xlabel("ω")
ax[1].set_ylabel("τ_g(ω)")
ax[1].legend()

plt.show()


## 5) Poles, zeros, and group delay

Geometric interpretation:

- Each **pole** near the unit circle creates a **positive peak** in group delay  
- Each **zero** near the unit circle creates a **negative dip** in group delay  

More precisely:

\[
\tau_g(\omega) = \sum_{\text{poles}} \tau_p(\omega) - \sum_{\text{zeros}} \tau_z(\omega)
\]

So:

- Poles cause energy to **linger** (resonance, ringing)  
- Zeros can cause **phase advance** or cancellation  

This directly links pole–zero geometry to **time-domain smearing**.


### Example: pole near unit circle ⇒ large group delay peak

In [ ]:
# IIR with pole near unit circle
a = 0.95
b = np.array([1.0])
# H(z) = 1 / (1 - a z^{-1})

H = 1 / (1 - a * z**-1)
phi = np.unwrap(np.angle(H))
tau = -np.gradient(phi, w)

fig, ax = plt.subplots(2, 1, figsize=(8,8), sharex=True)

ax[0].plot(w, np.abs(H))
ax[0].set_title("Magnitude response |H| (pole near unit circle)")
ax[0].set_ylabel("|H|")

ax[1].plot(w, tau)
ax[1].set_title("Group delay peak due to pole at 0.95")
ax[1].set_xlabel("ω")
ax[1].set_ylabel("τ_g(ω)")

plt.show()


## 6) Dispersion

In wave propagation and signal processing, **dispersion** means:

> Different frequencies propagate at **different speeds**.

Mathematically:

- Phase velocity: \(v_p = \omega / k\)  
- Group velocity: \(v_g = d\omega / dk\)  

In filters and systems:

- Non-constant group delay ⇒ **dispersive system**  
- Wave packets spread in time  
- Pulses distort

Linear phase ⇒ **no dispersion**.


## 7) Time-domain illustration: dispersion of a pulse

In [ ]:
# Narrowband pulse through dispersive vs nondispersive systems

# Input pulse
n = np.arange(-100, 101)
x = np.exp(-0.01*n**2) * np.cos(0.3*np.pi*n)

# Linear-phase FIR
b = b_lin
y_lin = np.convolve(x, b, mode='same')

# Dispersive IIR (pole near unit circle)
a = 0.95
y_iir = np.zeros_like(x)
for i in range(len(x)):
    y_iir[i] = x[i]
    if i-1 >= 0:
        y_iir[i] += a * y_iir[i-1]

fig, ax = plt.subplots(figsize=(9,5))
ax.plot(n, x, label="Input pulse")
ax.plot(n, y_lin, label="After linear-phase FIR")
ax.plot(n, y_iir, label="After dispersive IIR")
ax.set_title("Pulse distortion due to dispersion")
ax.set_xlabel("n")
ax.set_ylabel("Amplitude")
ax.legend()
plt.show()


## 8) Connection to seismology and wave physics

In physical wave propagation:

- Phase \(\phi(\omega)\) comes from the **dispersion relation**  
- Group delay corresponds to **travel time** of wave packets  
- Frequency-dependent group delay ⇒ **dispersive medium**  

Examples:

- Surface waves in layered Earth  
- Guided waves in crustal waveguides  
- Scattering and resonance near volcanoes  

In signal processing:

- Poles near unit circle ⇒ ringing, long coda  
- Non-minimum-phase zeros ⇒ pre-echo  
- Linear-phase filters preserve arrival times  


## 9) Final summary

| Concept | Meaning |
|--------|---------|
| Phase | Controls timing and waveform shape |
| Group delay | \(-d\phi/d\omega\), delay of energy |
| Linear phase | Constant group delay, no dispersion |
| Nonlinear phase | Frequency-dependent delay, dispersion |
| Poles near unit circle | Large group delay, ringing |
| Zeros near unit circle | Phase advance, notches |
| Dispersion | Pulse spreading due to frequency-dependent delay |

Key message:

> Magnitude shapes **how much** energy passes.  
> Phase shapes **when** that energy arrives.  
> Group delay tells you **how waveforms are distorted in time**.
